In [2]:
cd ../..

/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
# db engine
engine = create_engine('sqlite:///data/db/mirscribe.db')
pd.set_option('future.no_silent_downcasting', True)

# pyensembl db engine
pyensembl = create_engine('sqlite:////home/nazif/.cache/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.gtf.db')


In [4]:
genes = pd.read_sql("gene", pyensembl, columns=["gene_id", "gene_name", "seqname", "start", "end", "gene_biotype"])

genes = genes.replace(r'^\s*$', np.nan, regex=True)
genes = genes.infer_objects(copy=False)

genes.rename(columns={"seqname": "chr"}, inplace=True)

genes.head()


,gene_id,gene_name,chr,start,end,gene_biotype
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene


In [5]:
expected_chroms = [str(i) for i in range(1, 23)] + ['X', 'Y']
len_before = len(genes)
len_expected_chroms = len(expected_chroms)
# drop patch chromosomes
genes = genes[genes.chr.isin(expected_chroms)]

print(f"dropped {len_before - len(genes)} genes from patches")

dropped 5941 genes from patches


# add biomart data

In [6]:
e75 = pd.read_csv("data/biomart/ensembl75_g37.tsv", sep="\t")
colnames = {'Gene stable ID': 'gene_id',
            'EntrezGene ID': 'entrez_id',
            'Gene description': 'gene_description',
            'Gene name': 'gene_name',
            'Gene type': 'biomart_biotype',
            }

e75 = e75.rename(columns={"Ensembl Gene ID": "gene_id"})


e112 = pd.read_csv("data/biomart/ensembl112_g37.tsv", sep="\t")
colnames = {'Gene stable ID': 'gene_id',
            'NCBI gene (formerly Entrezgene) ID': 'entrez_id',
            'Gene description': 'gene_description',
            'Gene type': 'biomart_biotype',
            }

e112 = e112.rename(columns=colnames)

In [7]:
pyen = set(genes.gene_id.tolist())
e75s = set(e75.gene_id.tolist())
e112s = set(e112.gene_id.tolist())

# compare 
print(f"there are {len(pyen - e75s)} genes in pyensembl75 not in e75")
print(f"there are {len(e75s - pyen)} genes in e75 not in pyensembl75")
print(f"there are {len(pyen & e75s)} genes in both")
print("######################")
print(f"there are {len(pyen - e112s)} genes in pyensembl75 not in e112")
print(f"there are {len(e112s - pyen)} genes in e112 not in pyensembl75")
print(f"there are {len(pyen & e112s)} genes in both")

there are 0 genes in pyensembl75 not in e75
there are 6366 genes in e75 not in pyensembl75
there are 57736 genes in both
######################
there are 0 genes in pyensembl75 not in e112
there are 5941 genes in e112 not in pyensembl75
there are 57736 genes in both


whole pipeline is done with pyensembl75, therefore i must use biomart export from ensembl 112. They're fully compatible

In [8]:
e112.entrez_id.value_counts()

entrez_id
80864.0        20
259215.0       17
100169763.0    16
554313.0       16
8367.0         16
               ..
149837.0        1
200634.0        1
6718.0          1
29125.0         1
100653067.0     1
Name: count, Length: 25788, dtype: int64

In [9]:
e112_to_merge = e112[["gene_id","gene_description"]].drop_duplicates()
df = pd.merge(genes, e112_to_merge, on="gene_id", how="left")

df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233]
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ..."
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem..."


In [10]:
df.isna().sum()

gene_id                 0
gene_name               0
chr                     0
start                   0
end                     0
gene_biotype            0
gene_description    21297
dtype: int64

In [11]:
df["gene_description"] = df["gene_description"].fillna("no_description")


In [12]:
df.isna().sum()

gene_id             0
gene_name           0
chr                 0
start               0
end                 0
gene_biotype        0
gene_description    0
dtype: int64

# adding oncokb data

In [13]:
kb = pd.read_csv("data/oncokb/oncokb.csv", usecols=["gene_symbol", "ensembl_gene_id", "is_oncogene", "is_tumor_suppressor"])
kb = kb.rename(columns={"gene_symbol": "gene_name", "ensembl_gene_id": "gene_id", "is_oncogene": "is_oncogene_oncokb", "is_tumor_suppressor": "is_tsupp_oncokb"})

# drop columns where all bool values are false
cols = ["is_oncogene_oncokb", "is_tsupp_oncokb"]
kb = kb[~(kb[cols] == False).all(axis=1)]

kb.head()



,gene_name,gene_id,is_oncogene_oncokb,is_tsupp_oncokb
0,ABL1,ENSG00000097007,True,False
1,AKT1,ENSG00000142208,True,False
2,ALK,ENSG00000171094,True,False
3,AMER1,ENSG00000184675,False,True
4,APC,ENSG00000134982,False,True


# checking if all gene ids are found in our db

In [14]:
kb.gene_id.isin(df.gene_id).value_counts()

gene_id
True     721
False      4
Name: count, dtype: int64

In [15]:
missing_gene_ids = kb[~kb.gene_id.isin(df.gene_id)].gene_name
missing_gene_ids

238     IKBKE
394      CCNQ
453    H2AC17
697     GTF2I
Name: gene_name, dtype: object

In [16]:
missing_id_but_found_gene_names = df[df.gene_name.isin(missing_gene_ids)].gene_name
missing_id_but_found_gene_names

4487     IKBKE
22142    GTF2I
Name: gene_name, dtype: object

In [17]:
print(f"These genes are missing in the genes table: {list(set(missing_gene_ids) - set(missing_id_but_found_gene_names))}")

These genes are missing in the genes table: ['H2AC17', 'CCNQ']


H2AC17 is whole locus

H4C9 not found in grch37

H3C2 is HIST1H3D in grch37

IG's are whole loci with more than one ENSG ids

CCNQ is FAM58A in g37

note: these are found from g37 ensembl website

In [18]:
temp_df = kb.drop(columns=["gene_name"])
df = pd.merge(df, temp_df, on="gene_id", how="left")
df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene_oncokb,is_tsupp_oncokb
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,NaN,NaN
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,NaN,NaN
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],NaN,NaN
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",NaN,NaN
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",NaN,NaN


# adding details of genes above

In [19]:
kb[kb.gene_name.isin(missing_id_but_found_gene_names)]


,gene_name,gene_id,is_oncogene_oncokb,is_tsupp_oncokb
238,IKBKE,ENSG00000263528,True,False
697,GTF2I,ENSG00000263001,True,False


In [20]:
df[df.gene_name.isin(missing_id_but_found_gene_names)]

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene_oncokb,is_tsupp_oncokb
4487,ENSG00000143466,IKBKE,1,206643791,206670223,protein_coding,inhibitor of kappa light polypeptide gene enha...,NaN,NaN
22142,ENSG00000077809,GTF2I,7,74071994,74175026,protein_coding,general transcription factor IIi [Source:HGNC ...,NaN,NaN


In [21]:
df.loc[df.gene_name == 'IKBKE', 'is_oncogene_kb'] = True
df.loc[df.gene_name == 'GTF2I', 'is_oncogene_kb'] = True


In [22]:
kb[kb.gene_name == "CCNQ"]

,gene_name,gene_id,is_oncogene_oncokb,is_tsupp_oncokb
394,CCNQ,ENSG00000262919,False,True


In [23]:
df.loc[df.gene_name == 'FAM58A', 'is_tsupp_kb'] = True


# checking NA

In [24]:
df.isna().sum()

gene_id                   0
gene_name                 0
chr                       0
start                     0
end                       0
gene_biotype              0
gene_description          0
is_oncogene_oncokb    57015
is_tsupp_oncokb       57015
is_oncogene_kb        57734
is_tsupp_kb           57735
dtype: int64

In [25]:
df.fillna(False, inplace=True)
df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene_oncokb,is_tsupp_oncokb,is_oncogene_kb,is_tsupp_kb
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,False,False,False,False
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,False,False,False,False
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],False,False,False,False
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",False,False,False,False
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",False,False,False,False


# adding intogen data

In [26]:
into = pd.read_json("data/intogen/driver_genes.json")
into.rename(columns={"is_driver": "is_driver_intogen"}, inplace=True)
into.head()

,gene_name,is_driver_intogen
0,ACVR1,True
1,ACVR2A,True
2,ARAF,True
3,ASXL2,True
4,BAP1,True


In [27]:
into.gene_name.isin(df.gene_name).value_counts()

gene_name
True     86
False     1
Name: count, dtype: int64

In [28]:
into[~into.gene_name.isin(df.gene_name)].gene_name

78    TENT5C
Name: gene_name, dtype: object

FAM46C is the grch37 name of TENT5C

source: https://www.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000183508;r=1:117606048-117628389;t=ENST00000369448

In [29]:
df = pd.merge(df, into, on="gene_name", how="left")

In [30]:
df.loc[df.gene_name == "FAM46C", "is_driver_intogen"] = True

In [31]:
df["is_driver_intogen"] = df["is_driver_intogen"].fillna(False)

In [32]:
df.isna().sum()

gene_id               0
gene_name             0
chr                   0
start                 0
end                   0
gene_biotype          0
gene_description      0
is_oncogene_oncokb    0
is_tsupp_oncokb       0
is_oncogene_kb        0
is_tsupp_kb           0
is_driver_intogen     0
dtype: int64

# cosmic data

In [35]:
cs = (
    pd.read_csv("data/cosmic/Census_allTue Jun 4 15 09 41 2024.csv")
    .rename(columns={
        'Gene Symbol': 'gene_name',
        'Tier': 'tier_cosmic',
        'Hallmark': 'is_hallmark_cosmic',
        'Role in Cancer': 'role_in_cancer',
    }))


cs = cs[["gene_name", "tier_cosmic", "is_hallmark_cosmic", "role_in_cancer"]]

# drop rows where role_in_cancer is null
cs.dropna(subset=["role_in_cancer"], inplace=True)

# convert hallmark col into boolean
cs['is_hallmark_cosmic'] = cs['is_hallmark_cosmic'].replace({'Yes': True, np.nan: False})

cs['role_in_cancer'] = cs['role_in_cancer'].str.split(', ').apply(lambda x: [role.strip() for role in x])
unique_roles = {role for sublist in cs['role_in_cancer'] for role in sublist}

for role in unique_roles:
    cs[role] = cs['role_in_cancer'].apply(lambda x: role in x)

cs = cs.drop(columns=['role_in_cancer', "fusion"])

cs = cs.rename(columns={"TSG": "is_tumor_suppressor_cosmic", "oncogene": "is_oncogene_cosmic"})

cs


,gene_name,tier_cosmic,is_hallmark_cosmic,is_tumor_suppressor_cosmic,is_oncogene_cosmic
0,A1CF,2,False,False,True
1,ABI1,1,True,True,False
2,ABL1,1,True,False,True
3,ABL2,1,False,False,True
4,ACKR3,1,True,False,True
...,...,...,...,...,...
741,ZNF331,1,False,True,False
742,ZNF384,1,False,False,False
745,ZNF521,1,False,False,True
746,ZNRF3,2,False,True,False


In [36]:
cs.gene_name.isin(df.gene_name).value_counts()

gene_name
True     691
False     15
Name: count, dtype: int64

In [38]:
# List of gene names
gene_names = cs[~cs.gene_name.isin(df.gene_name)].gene_name

# Base URL for the search
base_url = "https://www.ensembl.org/Multi/Search/Results?q={};site=ensembl_all"

# Generate search URLs for each gene
search_urls = [base_url.format(gene) for gene in gene_names]

# Print the search URLs
for url in search_urls:
    print(url)


https://www.ensembl.org/Multi/Search/Results?q=AFDN;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=IGH;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=IGK;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=IGL;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=KNL1;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=LHFPL6;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=MRTFA;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=NSD2;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=NSD3;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=SHTN1;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=TENT5C;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=TRA;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=TRB;site=ensembl_all
https://www.ensembl.org/Multi/Search/Results?q=TRD;site=ensembl_all
https://www.ensembl.org/Multi/Sear

# manual checking genes above in g37

AFDN: MLLT4 

KNL1: CASC5

LHFPL6: LHFP

MRTFA: MKL1

NSD2: WHSC1

NSD3: WHSC1L1

SHTN1: KIAA1598

TENT5C: FAM46C

WDCP: C2orf44

immunoglobins are problematic in g37, discarded


TRAs are problematic



In [39]:
# Dictionary for replacements
replacements = {
    'AFDN': 'MLLT4',
    'KNL1': 'CASC5',
    'LHFPL6': 'LHFP',
    'MRTFA': 'MKL1',
    'NSD2': 'WHSC1',
    'NSD3': 'WHSC1L1',
    'SHTN1': 'KIAA1598',
    'TENT5C': 'FAM46C',
    'WDCP': 'C2orf44'
}

# Replace values in the 'gene_name' column
cs['gene_name'] = cs['gene_name'].replace(replacements)


In [40]:
df = pd.merge(df, cs, how='left', on='gene_name')
df["tier_cosmic"] = df["tier_cosmic"].fillna(0).astype(int)
df.fillna(False, inplace=True)

In [41]:
df.head()

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene_oncokb,is_tsupp_oncokb,is_oncogene_kb,is_tsupp_kb,is_driver_intogen,tier_cosmic,is_hallmark_cosmic,is_tumor_suppressor_cosmic,is_oncogene_cosmic
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,False,False,False,False,False,0,False,False,False
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,False,False,False,False,False,0,False,False,False
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],False,False,False,False,False,0,False,False,False
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",False,False,False,False,False,0,False,False,False
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",False,False,False,False,False,0,False,False,False


all dbs are merged on the top part

# post merge edits


In [42]:
df

,gene_id,gene_name,chr,start,end,gene_biotype,gene_description,is_oncogene_oncokb,is_tsupp_oncokb,is_oncogene_kb,is_tsupp_kb,is_driver_intogen,tier_cosmic,is_hallmark_cosmic,is_tumor_suppressor_cosmic,is_oncogene_cosmic
0,ENSG00000223972,DDX11L1,1,11869,14412,pseudogene,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...,False,False,False,False,False,0,False,False,False
1,ENSG00000227232,WASH7P,1,14363,29806,pseudogene,WAS protein family homolog 7 pseudogene [Sourc...,False,False,False,False,False,0,False,False,False
2,ENSG00000243485,MIR1302-10,1,29554,31109,lincRNA,microRNA 1302-10 [Source:HGNC Symbol;Acc:38233],False,False,False,False,False,0,False,False,False
3,ENSG00000237613,FAM138A,1,34554,36081,lincRNA,"family with sequence similarity 138, member A ...",False,False,False,False,False,0,False,False,False
4,ENSG00000268020,OR4G4P,1,52473,54936,pseudogene,"olfactory receptor, family 4, subfamily G, mem...",False,False,False,False,False,0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57731,ENSG00000272283,Metazoa_SRP,21,47955709,47956003,misc_RNA,Metazoan signal recognition particle RNA [Sour...,False,False,False,False,False,0,False,False,False
57732,ENSG00000160307,S100B,21,48018875,48025121,protein_coding,S100 calcium binding protein B [Source:HGNC Sy...,False,False,False,False,False,0,False,False,False
57733,ENSG00000160310,PRMT2,21,48055079,48085036,protein_coding,protein arginine methyltransferase 2 [Source:H...,False,False,False,False,False,0,False,False,False
57734,ENSG00000230982,DSTNP1,21,48073470,48073934,pseudogene,destrin (actin depolymerizing factor) pseudoge...,False,False,False,False,False,0,False,False,False


In [43]:
df["is_oncogene_consensus"] = df[['is_oncogene_kb', 'is_oncogene_cosmic']].any(axis=1)
df["is_tsupp_consensus"] = df[['is_tsupp_kb', 'is_tumor_suppressor_cosmic']].any(axis=1)

# df.drop(columns=['is_oncogene_kb', 'is_tsupp_kb', 'is_oncogene_cosmic', 'is_tumor_suppressor_cosmic'], inplace=True)

In [47]:
df['is_gene_of_interest'] = df[['is_oncogene_consensus', 'is_tsupp_consensus', 'is_driver_intogen', "is_hallmark_cosmic"]].any(axis=1)


In [51]:
df['cancer_gene_role'] = np.select(
    [
        (df['is_oncogene_consensus'] & df['is_tsupp_consensus']),
        (df['is_oncogene_consensus'] & ~df['is_tsupp_consensus']),
        (~df['is_oncogene_consensus'] & df['is_tsupp_consensus']),
        (~df['is_oncogene_consensus'] & ~df['is_tsupp_consensus'])
    ],
    ['dual_role', 'oncogene', 'tumor_suppressor', 'neither'],
    default='unknown'
)


# save to db

In [59]:
from sqlalchemy import create_engine

# db engine
engine = create_engine('sqlite:///data/db/mirscribe.db')

df.to_sql(name="genes", con=engine, if_exists="replace", index=False)

57736